# TAR: Taller de Aprendizaje por Refuerzo 2025
## Laboratorio 5: Gymnasium + stable_baselines3

### Stable Baselines3 (SB3) es una libreria algoritmos de reinforcement learning implementada en PyTorch. 

*Ejecutar esta celda solo la primera vez (si estan usando un entorno local - es la misma del lab 4) para descargar e instalar los paquetes necesarios. Si ejecutan el notebook en colab tendran que ejecutarla cada vez que reinicien el kernel*

#### IMPORTS

In [1]:
import gymnasium as gym
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

## Ejercicio 1. Lunar Landing

En este ejemplo entrenaremos un agente que aprenda a aterrizar correctamente una nave en la luna.
Doc del ambiente: https://gymnasium.farama.org/environments/box2d/lunar_lander/


#### 1.1 Estudiar el ambiente

In [2]:
# TODO Definir el ambiente `LunarLander-v2` e imprimir el tamaño del espacio de observación y del espacio de acciones.

env = gym.make("LunarLander-v3", render_mode="human")
env.reset()

tamaño_obs = env.observation_space.shape
print("Tamaño del espacio de observación: ", tamaño_obs)

tamaño_acciones = env.action_space.n
print("Tamaño del espacio de acción: ", tamaño_acciones)

/Users/fbello/personal/fing/tar/.venv/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


Tamaño del espacio de observación:  (8,)
Tamaño del espacio de acción:  4


El vector de **observación** corresponde a:

El estado es un vector de 8 dimensiones: las coordenadas del módulo lunar en x e y, sus velocidades lineales en x e y, su ángulo, su velocidad angular, y dos valores booleanos que representan si cada pata está en contacto con el suelo o no.



Por otro lado, el espacio de **acciones** es:

- 0: no hacer nada
- 1: encender motor de orientación izquierdo
- 2: encender motor principal
- 3: encender motor de orientación derecho

Función de **recompensa**:

  Después de cada paso, se otorga una recompensa. La recompensa total de un episodio es la suma de las recompensas de todos los pasos dentro de ese episodio.

Para cada paso, la recompensa:
- Se incrementa/disminuye mientras más cerca/lejos esté el módulo lunar de la plataforma de aterrizaje.
- Se incrementa/disminuye mientras más lento/rápido se mueva el módulo lunar.
- Se disminuye mientras más inclinado esté el módulo lunar (ángulo no horizontal).
- Se incrementa en 10 puntos por cada pata que esté en contacto con el suelo.
- Se disminuye en 0.03 puntos cada frame que un motor lateral esté encendido.
- Se disminuye en 0.3 puntos cada frame que el motor principal esté encendido.

El episodio recibe una recompensa adicional de -100 o +100 puntos por estrellarse o aterrizar de forma segura respectivamente.

Un episodio se considera una solución si obtiene al menos 200 puntos.

Un episodio termina si:
- El módulo lunar se estrella (el cuerpo del módulo entra en contacto con la luna)
- El módulo lunar sale del viewport (la coordenada x es mayor que 1)
- El módulo lunar no está despierto. Según la documentación de Box2D, un cuerpo que no está despierto es un cuerpo que no se mueve y no colisiona con ningún otro cuerpo

##### Multi-procesamiento
A continuación se vectoriza el ambiente para poder paralelizar el entrenamiento, (por suerte la libreria nos da las heramientas para lograr esto sin mucho esfuerzo)

In [3]:
from stable_baselines3.common.env_util import make_vec_env

# Create the environment
env = make_vec_env('LunarLander-v3', n_envs=16)

#### 1.2 Definir el modelo usando el modelo [Proximal Policy Optimization(PPO) de stable_baselines3](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html)

Leer la documentación y variar diferentes hiperparámetros como `batch_size`, `gamma`, entre otros.

¿Que tipo de metodo implementa?

#### RESPUESTA --> 

Es un modelo Policy Based, lo que significa que aprende directamente una política 

In [4]:
from stable_baselines3 import PPO


model = PPO(
    policy = 'MlpPolicy',
    env = env,
    n_steps = 1024,
    batch_size = 64,
    n_epochs = 4,
    gamma = 0.999,
    gae_lambda = 0.98,
    ent_coef = 0.01,
    device="cpu",
    verbose=1
)

Using cpu device


In [5]:
# Entrenar por 1.000.000 timesteps

model.learn(total_timesteps=1000000)

model_name = "ppo-LunarLander-v3"

# Guardar el modelo
model.save(model_name)


---------------------------------
| rollout/           |          |
|    ep_len_mean     | 92.8     |
|    ep_rew_mean     | -172     |
| time/              |          |
|    fps             | 12002    |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 16384    |
---------------------------------
------------------------------------------
| rollout/                |              |
|    ep_len_mean          | 97           |
|    ep_rew_mean          | -160         |
| time/                   |              |
|    fps                  | 8452         |
|    iterations           | 2            |
|    time_elapsed         | 3            |
|    total_timesteps      | 32768        |
| train/                  |              |
|    approx_kl            | 0.0053570154 |
|    clip_fraction        | 0.0204       |
|    clip_range           | 0.2          |
|    entropy_loss         | -1.38        |
|    explained_variance   | -0.00149     |
|    learning_r

#### 1.3 Evaluación del modelo:

¿Como podemos evaluar si el agente aprendió a aterrizar la nave?

##### RESPUESTA --> 

Como dice la documentación del ambiente, un episodio con una reward mayor a 200 se considera una solución

In [6]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(gym.make("LunarLander-v3", render_mode='rgb_array'))
mean_reward, std_reward = evaluate_policy(
    model, eval_env, n_eval_episodes=50, deterministic=True
)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=249.54 +/- 38.21892011518043


#### 1.4 Correr el agente entrenado y guardar un video del aterrizaje

si esto da error de timeout pueden generar un .py y correrlo desde consola para guardar el video

In [11]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import RecordVideo

# Load the trained model
model = PPO.load("ppo-LunarLander-v3")

# Create the environment with the proper render_mode for video recording
env = DummyVecEnv([lambda: RecordVideo(gym.make("LunarLander-v3", render_mode="rgb_array"), "./videos/PPO", episode_trigger=lambda x: True)])

# Reset the environment to get the initial observation
obs = env.reset()
done = False

# Play one episode to capture the video
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
# Close the environment to save the video file
env.close()

print("Video saved in the './videos' directory.")


/Users/fbello/personal/fing/tar/.venv/lib/python3.12/site-packages/gymnasium/wrappers/rendering.py:283: UserWarning: WARN: Overwriting existing videos at /Users/fbello/personal/fing/tar/videos/PPO folder (try specifying a different `video_folder` for the `RecordVideo` wrapper if this is not desired)
  logger.warn(


Video saved in the './videos' directory.


## Ejercicio 2. Cart Pole

En este ejemplo entrenaremos un agente que aprenda a equilibrar un poste sobre un carrito que se mueve sobre una linea.
Doc del ambiente: https://gymnasium.farama.org/environments/classic_control/cart_pole/


#### 2.1 Estudiar el ambiente:

¿Como son el espacio de acciones y el espacio de estados?

##### RESPUESTA -->

**Acciones:**
- 0: Empujar el carrito hacia la izquierda
- 1: Empujar el carrito hacia la derecha

**Estados:**

La observación es un ndarray con forma (4,) con los valores correspondientes a las siguientes posiciones y velocidades:

| Num | Observación | Mín | Máx |
|-----|-------------|-----|-----|
| 0 | Posición del Carrito | -4.8 | 4.8 |
| 1 | Velocidad del Carrito | -Inf | Inf |
| 2 | Ángulo del Poste | ~ -0.418 rad (-24°) | ~ 0.418 rad (24°) |
| 3 | Velocidad Angular del Poste | -Inf | Inf |

**Nota:** Aunque los rangos anteriores denotan los valores posibles para el espacio de observación de cada elemento, no refleja los valores permitidos del espacio de estados en un episodio no terminado. Particularmente:

- La posición x del carrito (índice 0) puede tomar valores entre (-4.8, 4.8), pero el episodio termina si el carrito sale del rango (-2.4, 2.4).
- El ángulo del poste puede observarse entre (-.418, .418) radianes (o ±24°), pero el episodio termina si el ángulo del poste no está en el rango (-.2095, .2095) (o ±12°).

In [12]:
# TODO Definir el ambiente e imprimir el tamaño del espacio de observación y del espacio de acciones.
import gymnasium as gym

env = gym.make("CartPole-v1", render_mode="rgb_array")

env.reset()

tamaño_obs = env.observation_space.shape 
print("Tamaño del espacio de observación: ", tamaño_obs)

tamaño_acciones = env.action_space.n 
print("Tamaño del espacio de acción: ", tamaño_acciones)

Tamaño del espacio de observación:  (4,)
Tamaño del espacio de acción:  2


#### 2.2 Definir el modelo usando el modelo - A2C de stable_baselines3

¿Que tipo de metodo implementa?

##### RESPUESTA --> 

In [13]:
from stable_baselines3 import A2C

model = A2C(
    policy="MlpPolicy",
    env=env,
    n_steps=5,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    device="cpu",
    verbose=1,
)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


#### 2.3 Vectorizar el ambiente y entrenar el modelo

In [14]:
from stable_baselines3.common.env_util import make_vec_env

env = make_vec_env("CartPole-v1", n_envs=16)

model = A2C(
    policy="MlpPolicy",
    env=env,
    n_steps=5,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    device="cpu",
    verbose=1,
)


model.learn(total_timesteps=1000000)

model_name = "A2C-CartPole-v1"

# Guardar el modelo
model.save(model_name)

Using cpu device
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 30.9     |
|    ep_rew_mean        | 30.9     |
| time/                 |          |
|    fps                | 23636    |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -0.618   |
|    explained_variance | 0.051    |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.744    |
|    value_loss         | 15.9     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 79.6     |
|    ep_rew_mean        | 79.6     |
| time/                 |          |
|    fps                | 26374    |
|    iterations         | 200      |
|    time_elapsed       | 0        |
|    total_timesteps    | 16000    |
| train/             

##### ¿Comó se si mi modelo aprendió?

Si la recompensa crece (es grande) (e.g. cerca de 500)

##### ¿Comó son las recompensas?

+1 por cada step

### RESPUESTA:

#### 2.4 Evaluar el agente


In [15]:
import gymnasium as gym
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from gymnasium.wrappers import RecordVideo

# Load the trained model
model = A2C.load("A2C-CartPole-v1")

# Create the environment with the proper render_mode for video recording
env = DummyVecEnv([lambda: RecordVideo(gym.make("CartPole-v1", render_mode="rgb_array"), "./videos/A2C", episode_trigger=lambda x: True)])

# Reset the environment to get the initial observation
obs = env.reset()
done = False

# Play one episode to capture the video
while not done:
    action, _ = model.predict(obs)
    obs, reward, done, info = env.step(action)
    
# Close the environment to save the video file
env.close()

print("Video saved in the './videos' directory.")

Video saved in the './videos' directory.


In [16]:
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

eval_env = Monitor(gym.make("CartPole-v1", render_mode="rgb_array"))
mean_reward, std_reward = evaluate_policy(
    model, eval_env, n_eval_episodes=50, deterministic=True
)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=500.00 +/- 0.0


: 